# Surface Classification

In [2]:
import igl # Read Triangular meshes
import pyacvd  # Remeshing
import numpy as np
import alphashape
import trimesh
import pymeshlab

# La py library for Laplacian 
import lapy as lp
from lapy import shapedna, plot, io
from scipy.sparse.linalg import inv


# Ploting libraries
import plotly.io as pio
pio.renderers.default = "sphinx_gallery"
import meshplot as mp # Visualization 

# Load local functions
from utils.utilities import *
from utils.icosphere import *

In [5]:
sv, sf = create_icosphere(subdivisions=6)
sv2, sf2 = create_icosphere(position=[4,0,0], radius=4,subdivisions=6)

sph = mp.plot(sv, sf, shading=shading)
sph.add_mesh(sv2, sf2, c=sv[:,1], shading=shading)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

## Icosphere Test

In [17]:
sv, sf = create_icosphere(1,6)

centers = circumcircle_faces(sv,sf)

print(sv[sf[:2],:])

#icosphere = mp.plot(sv,sf[:2], return_plot=True, shading=shading)

#icosphere.add_points(centers[:2],shading={"point_size": 0.05})



-0.014728847714849566
[[[-0.52573111  0.85065081  0.        ]
  [-0.53755701  0.84316611  0.01016771]
  [-0.52110531  0.85333381  0.0164517 ]]

 [[-0.53755701  0.84316611  0.01016771]
  [-0.54922203  0.83542908  0.02033237]
  [-0.53288296  0.84577011  0.02662092]]]


In [120]:
sv, sf = create_icosphere(1,6)


igl.write_obj("Ico.obj", sv, sf)

L = cotangent_matrix(sv, sf)
M = inv_mass_matrix(sv, sf)

Lap = M*L

Tmesh = lp.TriaMesh(sv,sf)
sol = shapedna.compute_shapedna(Tmesh, k=3)
eigs_lyp = sol["Eigenvalues"]

eigval, eigvec = eigs(Lap, 3)

[[4216.39652979 4216.39652979 4216.39652979 ... 5404.46063587
  5404.46063587 5404.46063587]]
TriaMesh with regular Laplace-Beltrami
Solver: spsolve (LU decomposition) ...


In [116]:
print(eigval)
print(eigs_lyp)

[-21926.56154562+0.j -21926.56154562+0.j -21926.56154562+0.j]
[-4.41938153e-14  1.03879708e-01  1.03879708e-01]


## Squirrel model

In [3]:
data_mesh = '/Users/cisneras/SGI Project/data/squirrel.stl'

# Load mesh
v, f = igl.read_triangle_mesh(data_mesh)

# Clean mesh
nv, nf = igl.remove_duplicates(v, f, 1e-6)

igl.write_obj("squirrel.obj",v,f)

#ratios = compute_ratio(v, f)

#idx = np.where(ratios > 3)

#  Visualization
#p = mp.plot(nv,nf, return_plot= True, shading=shading)
#p.add_mesh(nv, nf[idx], c=nv[:,2], shading=shading)


True

## Model Net 

### Alpha Shape

In [23]:
v, f =  igl.read_triangle_mesh("/Users/cisneras/SGI Project/data/ModelNet40/airplane/train/airplane_0099.off")
pts = [
    (0., 0., 0.), (0., 0., 1.), (0., 1., 0.),
    (1., 0., 0.), (1., 1., 0.), (1., 0., 1.),
    (0., 1., 1.), (1., 1., 1.), (.25, .5, .5),
    (.5, .25, .5), (.5, .5, .25), (.75, .5, .5),
    (.5, .75, .5), (.5, .5, .75)
]
alpha = alphashape.alphashape(v)

av = alpha.vertices
af = alpha.faces 

mp.plot(av,af)
#igl.write_obj("airplanealpha.obj",av,af)


KeyboardInterrupt: 

### Pymeshlab ball pivoting

In [33]:
v, f = trimesh.sample.sample_surface_even(air, 40*n, radius=None, seed=None)

alpha = alphashape.alphashape(v, 0.005)

av = alpha.vertices
af = alpha.faces 

#mp.plot(av,af, shading= shading)
igl.write_obj("airplanealpha.obj",av,af)
#mp.plot(v, shading={"point_size": 1.5})

True

In [45]:

m = pymeshlab.Mesh(np.array(v))


In [8]:
air = trimesh.load("/Users/cisneras/SGI Project/data/ModelNet40/airplane/train/airplane_0099.off")
#n = len(air.vertices)
#v, f = trimesh.sample.sample_surface_even(air, 40*n, radius=None, seed=None)
v = air.vertices
f = air.faces
m = pymeshlab.Mesh(np.array(v), np.array(f))

ms = pymeshlab.MeshSet()

#ms.load_new_mesh("/Users/cisneras/SGI Project/data/ModelNet40/airplane/train/airplane_0099.off")
ms.add_mesh(m, 'airplane_sampled')
ms.generate_surface_reconstruction_ball_pivoting(clustering= 20, creasethr=60)
ms.save_current_mesh("airplaneballpivot.obj")